In [1]:
# import os + os.chir is used to get to the llms directory to use the usermodules 
import os
os.chdir('/data/homezvol2/denisw1/copilot/llms/user_modules') 

from  table_instances import Summaries, Sentiments, Base
from  db import Session, get_training_data, add_summaries_data
from helper_functions import compute_summary_scores
import pandas as pd
import matplotlib.pyplot as plt

session = Session()

In [2]:
zephyr = pd.read_csv("/data/homezvol2/denisw1/copilot/Datasets/zephyr_summary_results.csv")
zephyr.head()

,dialogue_id,summary,time_taken,memory_usage
0,1,The conversation between a doctor and a patien...,0.630070,233.003457
1,2,The conversation is between a doctor (Dr. Pete...,0.534402,213.580879
2,3,Two people are searching for a set of keys tha...,0.534402,213.580879
3,4,The conversation revolves around a person (#Pe...,0.534402,213.580879
4,5,"Malik asks Wen to dance, but she declines beca...",0.534402,213.580879


In [3]:
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [ ]:
merged_df = pd.merge(training_data[["dialogue_id","actual_summary"]], zephyr, on="dialogue_id", how='inner')
merged_df.head()

In [ ]:
merged_df = merged_df.rename(columns={'summary': 'generated_summary', 'time_taken':'time_summary_taken', 'memory_usage':'memory_summary_usage'})
merged_df.head()

In [ ]:
merged_df["model_id"] = "zephyr 7b beta"
merged_df.head()

In [ ]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

merged_df = merged_df.apply(compute_summary_scores, axis=1)
merged_df.head()

In [ ]:
merged_df.to_csv("/data/homezvol2/denisw1/copilot/Datasets/zephyr_scores.csv", index = False)

In [ ]:
zephyr_scores = pd.read_csv("/data/homezvol2/denisw1/copilot/Datasets/zephyr_scores.csv")
zephyr_scores.head()

In [ ]:
merged_df = merged_df[['dialogue_id', 'model_id', 'generated_summary', 'rouge_1', 'rouge_2', 'rouge_l', 'bert_precision', 'bert_recall', 'bert_f1', 'meteor', 'memory_summary_usage', 'time_summary_taken']]
merged_df.head()

In [ ]:
add_summaries_data(merged_df)